# 加入物质对动量角度分辨的影响

In [37]:
import os, sys
import pandas as pd 
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from tqdm import tqdm
from itertools import product

import uproot

this_path = os.path.abspath('.')
if this_path not in sys.path:
    sys.path.append(this_path)

import warnings
warnings.filterwarnings('ignore')

In [38]:
# from single_pp_fixed_enery_and_direction_func import *

In [39]:
def readRootFiles(root_files):
    df_list = []
    for root_file in tqdm(root_files):
        try:
            df_list.append(readOneFile(root_file))
        except Exception as e:
            print(root_file, e)
    df = pd.concat(df_list)

    return df

def readOneFile(root_file):
    rf = uproot.open(root_file)
    tree = rf['RecInfo']

    keys = ['momentum', 'theta', 'phi', 'kal_p', 'kal_px', 'kal_py', 'kal_pz']

    df = pd.DataFrame()
    p_index = [np.argmax(p) if len(p) else None for p in tree['kal_p'].array()]
    for key in keys:
        tmp = tree[key].array()
        if '_p' in key:
            df[key] = [tmp[i][p_index[i]] if not p_index[i] is None else np.nan for i in range(len(tmp)) ] 
        else:
            df[key] = list(tmp)

    df['cos_theta'] = np.cos(df.theta)

    return df

In [40]:
def getDeltaAngle(df):
    mc_p = np.array(pthetaphi2Pxpypz(df['momentum'], df['theta'],df['phi']))
    px, py, pz = df['kal_px'], df['kal_py'], df['kal_pz']
    p = np.array([px, py, pz])
    return math.acos(p.dot(mc_p) / (np.linalg.norm(p) * np.linalg.norm(mc_p)))

def pxpypz2Theta(df):
    px, py, pz = df['kal_px'], df['kal_py'], df['kal_pz']
    return math.acos(pz / np.linalg.norm(np.array([px,py,pz])))

def fitNorm(data, nbins=100):
    mean, std = data.mean(), data.std()
    data = data.loc[
        (data > mean-2*std) &
        (data < mean+2*std)
    ]
    
    # plt.figure(figsize=(4,2))
    n, bins, _ = plt.hist(data ,nbins, density=True, facecolor = 'grey', alpha = 0.5, label='before')
    centers = (0.5*(bins[1:]+bins[:-1]))
    pars, cov = curve_fit(lambda x, mu, sig : norm.pdf(x, loc=mu, scale=sig), centers, n, p0=[mean,std])
    plt.plot(centers, norm.pdf(centers,*pars), 'k--',linewidth = 2, label='fit before',)
    plt.title('$\mu={:.4f}\pm{:.4f}$, $\sigma={:.4f}\pm{:.4f}$'.format(pars[0],np.sqrt(cov[0,0]), pars[1], np.sqrt(cov[1,1 ])))
    # plt.show()
    plt.close()

    return pars[0], cov[0,0]**0.5, pars[1], cov[1,1]**0.5

In [41]:
index = ['p_loc', 'p_loc_err', 'p_scale', 'p_scale_err', 'theta_loc', 'theta_loc_err', 'theta_scale', 'theta_scale_err', 'efficiency', 'valid', 'all', 'delta_angle_one_sigma']

def df2Res(df):
    global index
    raw_shape = df.shape[0]
    df = df.loc[df.kal_p > 1e-6]
    efficiency = df.shape[0] / raw_shape if raw_shape else 0

    if not df.shape[0]:
        return pd.Series([np.nan]*len(index), index=index)

    df = df.copy()
    df['kal_theta'] = df.apply(pxpypz2Theta, axis=1)
    df['dp'] = df.kal_p - df.momentum
    df['dtheta'] = df.kal_theta - df.theta
    try:
        p_loc, p_loc_err, p_scale, p_scale_err = fitNorm(df['dp'])
        theta_loc, theta_loc_err, theta_scale, theta_scale_err = fitNorm(df['dtheta'])
    except:
        p_loc, p_loc_err, p_scale, p_scale_err, theta_loc, theta_loc_err, theta_scale, theta_scale_err = tuple([np.nan]*8)

    df['delta_angle'] = df.apply(getDeltaAngle, axis=1)
    delta_angle_one_sigma = np.percentile(df['delta_angle'].dropna(), 68.3)

    return pd.Series([p_loc, p_loc_err, p_scale, p_scale_err, theta_loc, theta_loc_err, theta_scale, theta_scale_err, efficiency, df.shape[0], raw_shape, delta_angle_one_sigma], index=index)



In [42]:
cos_theta_range = np.arange(0, 1, 0.1)

res_df = pd.DataFrame(columns=['particle', 'material', 'thickness', 'p', 'cos_theta'] + index)

for root_path in tqdm(glob(os.path.join(this_path, '../../data/gen/*p*/random/*/*'))):
    root_path_copy = root_path
    root_path_split = root_path_copy.replace('\\', '/').split('/')
    particle, material, thickness = root_path_split[-4], root_path_split[-2].split('_')[1], int(root_path_split[-1])

    p_min = 0.1 if 'pi' in particle else 0.3
    p_range = np.arange(p_min, 1.4, 0.1)

    # print(glob(os.path.join(root_path, '*.root')))
    # continue

    df = readRootFiles(glob(os.path.join(root_path, '*.root')))
    
    for i,j in tqdm(product(range(len(p_range)-1), range(len(cos_theta_range)-1))):
        p1, p2 = p_range[i], p_range[i+1]
        cos_theta1, cos_theta2 = cos_theta_range[j], cos_theta_range[j+1]

        tmp = df.loc[
            (df.momentum >= p1) &
            (df.momentum < p2) &
            (df.cos_theta >= cos_theta1) &
            (df.cos_theta < cos_theta2)
        ]
        res = df2Res(tmp)
        res = [particle, material, thickness, (p1+p2)/2, (cos_theta1+cos_theta2)/2] + list(res)
        res_df.loc[res_df.shape[0]] = res



  0%|          | 0/24 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
res_df.to_csv('single_ppi.csv')